<a href="https://colab.research.google.com/github/OziomaEunice/Sentiment_GPT/blob/develop/SVM_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SVM for Sentiment Analysis**

In [1]:
# install the necessary libraries
! pip install numpy
! pip install pandas
! pip install scikit-learn
! pip install nltk
! pip install openpyxl # for reading excel files

In [2]:
# import the necessary libraries
import numpy as np
import pandas as pd
import nltk
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [3]:
# Download the stopwords dataset
nltk.download('stopwords')

# Download wordnet dataset
nltk.download('wordnet')

# Download punkt dataset
nltk.download('punkt')

# Get the set of English stop words
stop_words = set(stopwords.words('english'))

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# since dataset is imported to my Google Colab (which will remain in here temporary)
# read csv file
df = pd.read_excel('/content/Tweets.xlsx')

In [5]:
# show dataframe
df.head(10)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760512,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:00,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:00,NaN,Pacific Time (US & Canada)
2,570301083672813568,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:00,Lets Play,Central Time (US & Canada)
3,570301031407624192,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:00,NaN,Pacific Time (US & Canada)
4,570300817074462720,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:00,NaN,Pacific Time (US & Canada)
5,570300767074181120,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:00,NaN,Pacific Time (US & Canada)
6,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:00,San Francisco CA,Pacific Time (US & Canada)
7,570300248553349120,neutral,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,@VirginAmerica Really missed a prime opportuni...,NaN,2015-02-24 11:12:00,Los Angeles,Pacific Time (US & Canada)
8,570299953286942720,positive,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",NaN,2015-02-24 11:11:00,San Diego,Pacific Time (US & Canada)
9,570295459631263744,positive,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:00,Los Angeles,Eastern Time (US & Canada)


In [6]:
# datatype info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   tweet_id                      14640 non-null  int64         
 1   airline_sentiment             14640 non-null  object        
 2   airline_sentiment_confidence  14640 non-null  float64       
 3   negativereason                9178 non-null   object        
 4   negativereason_confidence     10522 non-null  float64       
 5   airline                       14640 non-null  object        
 6   airline_sentiment_gold        40 non-null     object        
 7   name                          14640 non-null  object        
 8   negativereason_gold           32 non-null     object        
 9   retweet_count                 14640 non-null  int64         
 10  text                          14640 non-null  object        
 11  tweet_coord                 

## **Preprocessing Dataset**

In [7]:
# clean (preprocess) the dataset
def cleanData(text, min_word_length = 3):
  text = text.lower()
  text = re.sub(r'@[A-Za-z0-9]+', "", text) # this informs Python the the mentions in text must be substituted with an empty string
  text = re.sub(r'#', "", text) # removing #
  text = re.sub(r'RT[\s]+', "", text) # removing retweets
  text = re.sub(r'https?:\/\/\S+', "", text) # removing links
  text = ' '.join(word for word in text.split() if len(word) >= min_word_length and word not in stop_words) # Remove short words

  punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
  for p in punctuations:
      text = text.replace(p,'') #Removing punctuations

  # Lemmatize the words
  words = nltk.word_tokenize(text)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  text = ' '.join(lemmatized_words)

  return text

In [8]:
# df['text'] = df['text'].apply(cleanData)

# Apply the cleanData function to all text columns in the dataframe
df = df.applymap(lambda x: cleanData(x) if isinstance(x, str) else x)

In [9]:
df[['tweet_id', 'airline_sentiment', 'text']].head(20)

,tweet_id,airline_sentiment,text
0,570306133677760512,neutral,said
1,570301130888122368,positive,plus added commercial experience tacky
2,570301083672813568,neutral,today must mean need take another trip
3,570301031407624192,negative,really aggressive blast obnoxious `` entertain...
4,570300817074462720,negative,really big bad thing
5,570300767074181120,negative,seriously would pay 30 flight seat playing rea...
6,570300616901320704,positive,"yes , nearly every time fly “ ear worm ” won ’..."
7,570300248553349120,neutral,really missed prime opportunity men without ha...
8,570299953286942720,positive,"well , didnt…but do d"
9,570295459631263744,positive,"amazing , arrived hour early good me"


### **Converting labels into integers or numeric values for ML**
**The purpose of this conversion is so that machines can understand the labels, since they aren't able to understand text data**

In [10]:
# create a new column called data and link the airline_sentiment to it
# (this 'label' column will have numbers indicating 1, 0, -1
# (positive, neutral, negative)
df['label'] = df['airline_sentiment'].apply(lambda x: int(1) if str(x) == 'positive' else (0 if x == 'neutral' else -1) )

In [11]:
df[['tweet_id', 'airline_sentiment', 'text', 'label']].head(20)

,tweet_id,airline_sentiment,text,label
0,570306133677760512,neutral,said,0
1,570301130888122368,positive,plus added commercial experience tacky,1
2,570301083672813568,neutral,today must mean need take another trip,0
3,570301031407624192,negative,really aggressive blast obnoxious `` entertain...,-1
4,570300817074462720,negative,really big bad thing,-1
5,570300767074181120,negative,seriously would pay 30 flight seat playing rea...,-1
6,570300616901320704,positive,"yes , nearly every time fly “ ear worm ” won ’...",1
7,570300248553349120,neutral,really missed prime opportunity men without ha...,0
8,570299953286942720,positive,"well , didnt…but do d",1
9,570295459631263744,positive,"amazing , arrived hour early good me",1


## **Feature Extraction: TF-IDF Vectorisation**

### **Split the Dataset into Training and Testing**

In [12]:
text_train, text_test, label_train, label_test = train_test_split(df['text'].values, df['label'].values, test_size = 0.2, random_state = 42, stratify=df['label'].values)

print(len(text_train), len(text_test), len(label_train) , len(label_test))

11712 2928 11712 2928


### **TF-IDF**

In [13]:
# initialise the tf-idf vectoriser (or model) and store it in a variable
tfidf_vector = TfidfVectorizer()

# Fit and transform the entire dataset
tfidf_matrix = tfidf_vector.fit_transform(df['text'])

# Display the TF-IDF vocabulary
print("Vocabulary:", tfidf_vector.vocabulary_)

# Place the TF-IDF matrix into a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.todense(), columns=sorted(tfidf_vector.vocabulary_))

# Display the TF-IDF DataFrame
print(tfidf_df)

# Transform the training and testing sets
tfidf_train = tfidf_vector.transform(text_train)
tfidf_test = tfidf_vector.transform(text_test)

Vocabulary: {'said': 10537, 'plus': 9477, 'added': 1893, 'commercial': 3731, 'experience': 5246, 'tacky': 11679, 'today': 12039, 'must': 8392, 'mean': 8026, 'need': 8493, 'take': 11691, 'another': 2226, 'trip': 12202, 'really': 9964, 'aggressive': 1977, 'blast': 2884, 'obnoxious': 8811, 'entertainment': 5048, 'guest': 6246, 'face': 5307, 'amp': 2161, 'little': 7661, 'recourse': 10022, 'big': 2832, 'bad': 2603, 'thing': 11896, 'seriously': 10776, 'would': 13345, 'pay': 9234, '30': 803, 'flight': 5547, 'seat': 10687, 'playing': 9439, 'flying': 5683, 'yes': 13434, 'nearly': 8490, 'every': 5149, 'time': 11986, 'fly': 5672, 'ear': 4856, 'worm': 13315, 'won': 13276, 'away': 2531, 'missed': 8230, 'prime': 9675, 'opportunity': 8958, 'men': 8078, 'without': 13246, 'hat': 6351, 'parody': 9173, 'there': 11871, 'well': 13075, 'didnt': 4509, 'but': 3182, 'do': 4680, 'amazing': 2130, 'arrived': 2363, 'hour': 6613, 'early': 4859, 'good': 6099, 'me': 8021, 'know': 7368, 'suicide': 11532, 'second': 107

## **Training SVM**

In [14]:
# train the SVM model
svm = SVC(kernel='linear')

svm.fit(tfidf_train, label_train)

SVC(kernel='linear')

## **Testing SVM**

In [15]:
# Test the SVM model
predictions = svm.predict(tfidf_test)

print(classification_report(label_test,predictions))

              precision    recall  f1-score   support

          -1       0.81      0.92      0.86      1835
           0       0.66      0.52      0.58       620
           1       0.79      0.60      0.69       473

    accuracy                           0.78      2928
   macro avg       0.75      0.68      0.71      2928
weighted avg       0.78      0.78      0.77      2928

